In [12]:
import glob, os, shutil
import csv
import xml.etree.ElementTree as ET
import json
from openpyxl import Workbook
import pandas as pd
import pymysql
from openpyxl import Workbook
from openpyxl import load_workbook

# 항공
main_path = 'C:/Users/S2_SVR/Desktop/항공'
# 공사
# main_path = 'F:/작업폴더'
#화재
# main_path = 'C:/Users/S2_SVR/Desktop/화재'

os.chdir(main_path)

# 폴더 리스트 들고오기
path = "./"
aa = os.listdir(path)

file_list = [file for file in aa if file.endswith('.xlsx')]
print(file_list)

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

createFolder(main_path+'/csv파일')
createFolder(main_path+'/완료')

['[216]201202 한서대 비행교육장 N01216 (충남 태안군 남면 신온리 산105).xlsx', '[220]201202 한서대 비행교육장 N01220 (충남 태안군 남면 신온리 산105).xlsx', '[223]201020 한서대 비행교육장 N01223 (충남 태안군 남면 신온리 산105).xlsx', '[226]191020 한서대 비행교육장 O0101 (충남 태안군 남면 신온리 산105).xlsx', '[228]201201 한서대 비행교육장 N01228 (충남 태안군 남면 신온리 산105).xlsx', '[230]201022 한서대 비행교육장 N01230 (충남 태안군 남면 신온리 산105).xlsx', '[233]201201 한서대 비행교육장 N01233 (충남 태안군 남면 신온리 산105).xlsx', '[236]201201 한서대 비행교육장 N01236 (충남 태안군 남면 신온리 산105).xlsx', '[239]201201 한서대 비행교육장 N01239 (충남 태안군 남면 신온리 산105).xlsx', '[631]210105 스튜디오 N0661 (연출에 의한 촬영(20% 이내)).xlsx', '[635]210105 스튜디오 N0665 (연출에 의한 촬영(20% 이내)).xlsx', '[636]210105 스튜디오 N0666 (연출에 의한 촬영(20% 이내)).xlsx', '[637]210105 스튜디오 N0667 (연출에 의한 촬영(20% 이내)).xlsx']


In [13]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
createFolder(main_path+'/csv파일/BB')

# Making DB columns(공사)
def construct_BB(data):
        class_col = ['filename']

        for i in range(1, 46):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')
#         class_col.append('PO')
#         class_col.append('KP')

        construct_BB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            construct_BB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :
                if row['class_count'] == row['box'] :
                    construct_BB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']
                    construct_BB.loc[index, 'BB'] = row['box']
                
                else :
                    construct_BB.loc[index, 'class0{}'.format(row['class'])] = row['box']
                    construct_BB.loc[index, 'BB'] = row['box']

            else :
                 if row['class_count'] == row['box']:
                    construct_BB.loc[index, 'class{}'.format(row['class'])] = row['class_count']
                    construct_BB.loc[index, 'BB'] = row['box']

                 else :
                    construct_BB.loc[index, 'class{}'.format(row['class'])] = row['box']
                    construct_BB.loc[index, 'BB'] = row['box']


        # 빈 값은 0으로 대체
            construct_BB.fillna(0,inplace = True)
        
        construct_BB = construct_BB.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        construct_BB[construct_BB.columns.difference(['filename'])] = construct_BB[construct_BB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        construct_BB.to_csv(main_path+ '/csv파일/BB/' +'{}.csv'.format(file_name.split('.xlsx')[0], header = True, encoding = 'utf-8-sig', index = False))
        
        return construct_BB

 # Making DB columns(화재)
def fire_BB(data):
        class_col = ['filename']

        for i in range(1, 12):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')
#         class_col.append('PO')
#         class_col.append('KP')

        fire_BB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            fire_BB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :
                if row['class_count'] == row['box'] :
                    fire_BB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']
                    fire_BB.loc[index, 'BB'] = row['box']
                
                else :
                    fire_BB.loc[index, 'class0{}'.format(row['class'])] = row['box']
                    fire_BB.loc[index, 'BB'] = row['box']

            else :
                 if row['class_count'] == row['box']:
                    fire_BB.loc[index, 'class{}'.format(row['class'])] = row['class_count']
                    fire_BB.loc[index, 'BB'] = row['box']

                 else :
                    fire_BB.loc[index, 'class{}'.format(row['class'])] = row['box']
                    fire_BB.loc[index, 'BB'] = row['box']

        # 빈 값은 0으로 대체
            fire_BB.fillna(0,inplace = True)
            
        fire_BB = fire_BB.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        fire_BB[fire_BB.columns.difference(['filename'])] = fire_BB[fire_BB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        fire_BB.to_csv(main_path+ '/csv파일/BB/' +'{}.csv'.format(file_name.split('.xlsx')[0], header = True, encoding = 'utf-8-sig', index = False))
    
        return fire_BB

# Making DB columns(항공)
def airport_BB(data):
    
        class_col = ['filename']
        for i in range(1, 23):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')

        airport_BB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

#             airport_DB.loc[index, '_id'] = index
            airport_BB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :

                airport_BB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    airport_BB.loc[index, 'BB'] = row['box']

            else :
                airport_BB.loc[index, 'class{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    airport_BB.loc[index, 'BB'] = row['box']


        # 빈 값은 0으로 대체
            airport_BB.fillna(0,inplace = True)
            
        airport_BB = airport_BB.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        airport_BB[airport_BB.columns.difference(['filename'])] = airport_BB[airport_BB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        airport_BB.to_csv(main_path+ '/csv파일/BB/' +'{}.csv'.format(file_name.split('.xlsx')[0], header = True, encoding = 'utf-8-sig', index = False))
        
        return airport_BB

# 공사 DB
def insert_construct_bb(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "replace into s2_bb (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, class12, class13, class14, class15, class16, class17, class18, class19, class20, class21, class22, class23, class24, class25, class26, class27, class28, class29, class30, class31, class32, class33, class34, class35, class36, class37, class38, class39, class40, class41, class42, class43, class44, class45, BB) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

# 화재 DB
def insert_fire_bb(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s3_bb (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, BB) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) "
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()
            
# 항공 DB
def insert_airport_bb(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s1 (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, class12, class13, class14, class15, class16, class17, class18, class19, class20, class21, class22, BB) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()
            
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
createFolder(main_path+'/csv파일/PO')

# Making DB columns(공사)
def construct_PO(data):
        class_col = ['filename']

        for i in range(1, 46):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
#         class_col.append('BB')
        class_col.append('PO')
#         class_col.append('KP')

        construct_PO = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            construct_PO.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :
                if row['class_count'] == row['box'] :
                    construct_PO.loc[index, 'class0{}'.format(row['class'])] = 0
                    construct_PO.loc[index, 'PO'] = 0
                
                else :
                    construct_PO.loc[index, 'class0{}'.format(row['class'])] = row['polygon']
                    construct_PO.loc[index, 'PO'] = row['polygon']

            else :
                 if row['class_count'] == row['box']:
                    construct_PO.loc[index, 'class{}'.format(row['class'])] = 0
                    construct_PO.loc[index, 'PO'] = 0

                 else :
                    construct_PO.loc[index, 'class{}'.format(row['class'])] = row['polygon']
                    construct_PO.loc[index, 'PO'] = row['polygon']


        # 빈 값은 0으로 대체
            construct_PO.fillna(0,inplace = True)
        
        construct_PO = construct_PO.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        construct_PO[construct_PO.columns.difference(['filename'])] = construct_PO[construct_PO.columns.difference(['filename'])].astype('int64')
        # csv 저장
        construct_PO.to_csv(main_path+ '/csv파일/PO/' +'{}PO.csv'.format(file_name.split('.xlsx')[0], header = True, encoding = 'utf-8-sig', index = False))
        
        return construct_PO

 # Making DB columns(화재)
def fire_PO(data):
        class_col = ['filename']

        for i in range(1, 12):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
#         class_col.append('BB')
        class_col.append('PO')
#         class_col.append('KP')

        fire_PO = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            fire_PO.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :
                if row['class_count'] == row['box'] :
                    fire_PO.loc[index, 'class0{}'.format(row['class'])] = 0
                    fire_PO.loc[index, 'PO'] = 0
                
                else :
                    fire_PO.loc[index, 'class0{}'.format(row['class'])] = row['polygon']
                    fire_PO.loc[index, 'PO'] = row['polygon']
                    
            else :
                 if row['class_count'] == row['box']:
                    fire_PO.loc[index, 'class{}'.format(row['class'])] = 0
                    fire_PO.loc[index, 'PO'] = 0

                 else :
                    fire_PO.loc[index, 'class{}'.format(row['class'])] = row['polygon']
                    fire_PO.loc[index, 'PO'] = row['polygon']

        # 빈 값은 0으로 대체
            fire_PO.fillna(0,inplace = True)
            
        fire_PO = fire_PO.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        fire_PO[fire_PO.columns.difference(['filename'])] = fire_PO[fire_PO.columns.difference(['filename'])].astype('int64')
        # csv 저장
        fire_PO.to_csv(main_path+ '/csv파일/PO/' +'{}PO.csv'.format(file_name.split('.xlsx')[0], header = True, encoding = 'utf-8-sig', index = False))
    
        return fire_PO   

# 공사 DB
def insert_construct_po(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "replace into s2_po (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, class12, class13, class14, class15, class16, class17, class18, class19, class20, class21, class22, class23, class24, class25, class26, class27, class28, class29, class30, class31, class32, class33, class34, class35, class36, class37, class38, class39, class40, class41, class42, class43, class44, class45, PO) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

# 화재 DB
def insert_fire_po(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s3_po (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, PO) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) "
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

In [14]:
# file_name = file_list

# sample = pd.read_excel(file_name)
# sample['filename'] = sample['JSON']
# del sample['폴더명']
# del sample['JSON']
# sample = sample[['filename', 'class', 'class_count', 'box', 'polygon', 'point']]

# display(sample.isnull().sum())

# sample.fillna(method='ffill',inplace = True)
# sample = sample.astype({'filename':'object', 'class':'int64', 'class_count':'int64', 'box':'int64', 'polygon':'int64', 'point':'int64'})
# insert_fire_bb(fire_BB(sample))
# insert_fire_po(fire_PO(sample))

In [15]:
for i, file_name in enumerate(file_list):
    print(i)
    sample = pd.read_excel('{}'.format(file_name))
    sample['filename'] = sample['JSON']
    del sample['폴더명']
    del sample['JSON']
    sample = sample[['filename', 'class', 'class_count', 'box', 'polygon', 'point']]

    display(sample.isnull().sum())

    sample.fillna(method='ffill',inplace = True)
    sample = sample.astype({'filename':'object', 'class':'int64', 'class_count':'int64', 'box':'int64', 'polygon':'int64', 'point':'int64'})
    
    # 항공
    insert_airport_bb(airport_BB(sample))

    # 화재
    # insert_fire_bb(fire_BB(sample))
    # insert_fire_po(fire_PO(sample))

    # 공사
    # insert_construct_bb(construct_BB(sample))
    # insert_construct_po(construct_PO(sample))
    
    shutil.move(file_name, '완료/')

0


filename       6256
class             0
class_count       0
box               0
polygon           0
point             0
dtype: int64

1


filename       3537
class             0
class_count       0
box               0
polygon           0
point             0
dtype: int64

2


...

filename       4199
class             0
class_count       0
box               0
polygon           0
point             0
dtype: int64

11


filename       0
class          0
class_count    0
box            0
polygon        0
point          0
dtype: int64

12


filename       0
class          0
class_count    0
box            0
polygon        0
point          0
dtype: int64

### BB, PO 카운팅

In [5]:
write_xl = Workbook()
write_ws = write_xl.active
write_ws.append(['Folder', 'Image', 'BB', 'polygon'])

bb_list = os.listdir('F:/작업폴더/카운트/csv파일/BB')
po_list = os.listdir('F:/작업폴더/카운트/csv파일/PO')

for i in range(len(bb_list)):
    globals()[bb_list[i]] = pd.read_csv('F:/작업폴더/카운트/csv파일/BB/' + bb_list[i], index_col=0)
    globals()[po_list[i]] = pd.read_csv('F:/작업폴더/카운트/csv파일/PO/' + po_list[i], index_col=0)
    
    write_ws.append([bb_list[i].split('.csv')[0], len(globals()[bb_list[i]]), globals()[bb_list[i]]['BB'].sum(), globals()[po_list[i]]['PO'].sum()])

write_xl.save('Y:/양식/공사_폴더별_개수.xlsx')
print('완료')

완료


In [11]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
createFolder(main_path+'/csv파일/BB')

# Making DB columns(공사)
def construct_BB(data):
        class_col = ['filename']

        for i in range(1, 46):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')
#         class_col.append('PO')
#         class_col.append('KP')

        construct_BB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            construct_BB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :
                if row['class_count'] == row['box'] :
                    construct_BB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']
                    construct_BB.loc[index, 'BB'] = row['box']
                
                else :
                    construct_BB.loc[index, 'class0{}'.format(row['class'])] = row['box']
                    construct_BB.loc[index, 'BB'] = row['box']

            else :
                 if row['class_count'] == row['box']:
                    construct_BB.loc[index, 'class{}'.format(row['class'])] = row['class_count']
                    construct_BB.loc[index, 'BB'] = row['box']

                 else :
                    construct_BB.loc[index, 'class{}'.format(row['class'])] = row['box']
                    construct_BB.loc[index, 'BB'] = row['box']


        # 빈 값은 0으로 대체
            construct_BB.fillna(0,inplace = True)
        
        construct_BB = construct_BB.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        construct_BB[construct_BB.columns.difference(['filename'])] = construct_BB[construct_BB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        construct_BB.to_csv(main_path+ '/csv파일/BB/' +'{}.csv'.format(file_name, header = True, encoding = 'utf-8-sig', index = False))
        
        return construct_BB

 # Making DB columns(화재)
def fire_BB(data):
        class_col = ['filename']

        for i in range(1, 12):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')
#         class_col.append('PO')
#         class_col.append('KP')

        fire_BB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            fire_BB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :
                if row['class_count'] == row['box'] :
                    fire_BB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']
                    fire_BB.loc[index, 'BB'] = row['box']
                
                else :
                    fire_BB.loc[index, 'class0{}'.format(row['class'])] = row['box']
                    fire_BB.loc[index, 'BB'] = row['box']

            else :
                 if row['class_count'] == row['box']:
                    fire_BB.loc[index, 'class{}'.format(row['class'])] = row['class_count']
                    fire_BB.loc[index, 'BB'] = row['box']

                 else :
                    fire_BB.loc[index, 'class{}'.format(row['class'])] = row['box']
                    fire_BB.loc[index, 'BB'] = row['box']

        # 빈 값은 0으로 대체
            fire_BB.fillna(0,inplace = True)
            
        fire_BB = fire_BB.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        fire_BB[fire_BB.columns.difference(['filename'])] = fire_BB[fire_BB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        fire_BB.to_csv(main_path+ '/csv파일/BB/' +'{}.csv'.format(file_name, header = True, encoding = 'utf-8-sig', index = False))
    
        return fire_BB

# Making DB columns(항공)
def airport_BB(data):
    
        class_col = ['filename']
        for i in range(1, 23):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')

        airport_BB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

#             airport_DB.loc[index, '_id'] = index
            airport_BB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :

                airport_BB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    airport_BB.loc[index, 'BB'] = row['box']

            else :
                airport_BB.loc[index, 'class{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    airport_BB.loc[index, 'BB'] = row['box']


        # 빈 값은 0으로 대체
            airport_BB.fillna(0,inplace = True)
            
        airport_BB = airport_BB.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        airport_BB[airport_BB.columns.difference(['filename'])] = airport_BB[airport_BB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        airport_BB.to_csv(main_path+ '/csv파일/BB/' +'{}.csv'.format(file_name, header = True, encoding = 'utf-8-sig', index = False))
        
        return airport_BB

# 공사 DB
def insert_construct_bb(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s2_bb (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, class12, class13, class14, class15, class16, class17, class18, class19, class20, class21, class22, class23, class24, class25, class26, class27, class28, class29, class30, class31, class32, class33, class34, class35, class36, class37, class38, class39, class40, class41, class42, class43, class44, class45, BB) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

# 화재 DB
def insert_fire_bb(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s3_bb (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, BB) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) "
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()
            
# 항공 DB
def insert_airport_bb(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s1 (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, class12, class13, class14, class15, class16, class17, class18, class19, class20, class21, class22, BB) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

### PO저장

In [12]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 
createFolder(main_path+'/csv파일/PO')

# Making DB columns(공사)
def construct_PO(data):
        class_col = ['filename']

        for i in range(1, 46):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
#         class_col.append('BB')
        class_col.append('PO')
#         class_col.append('KP')

        construct_PO = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            construct_PO.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :
                if row['class_count'] == row['box'] :
                    construct_PO.loc[index, 'class0{}'.format(row['class'])] = 0
                    construct_PO.loc[index, 'PO'] = 0
                
                else :
                    construct_PO.loc[index, 'class0{}'.format(row['class'])] = row['polygon']
                    construct_PO.loc[index, 'PO'] = row['polygon']

            else :
                 if row['class_count'] == row['box']:
                    construct_PO.loc[index, 'class{}'.format(row['class'])] = 0
                    construct_PO.loc[index, 'PO'] = 0

                 else :
                    construct_PO.loc[index, 'class{}'.format(row['class'])] = row['polygon']
                    construct_PO.loc[index, 'PO'] = row['polygon']


        # 빈 값은 0으로 대체
            construct_PO.fillna(0,inplace = True)
        
        construct_PO = construct_PO.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        construct_PO[construct_PO.columns.difference(['filename'])] = construct_PO[construct_PO.columns.difference(['filename'])].astype('int64')
        # csv 저장
        construct_PO.to_csv(main_path+ '/csv파일/PO/' +'{}PO.csv'.format(file_name, header = True, encoding = 'utf-8-sig', index = False))
        
        return construct_PO

 # Making DB columns(화재)
def fire_PO(data):
        class_col = ['filename']

        for i in range(1, 12):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
#         class_col.append('BB')
        class_col.append('PO')
#         class_col.append('KP')

        fire_PO = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            fire_PO.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :
                if row['class_count'] == row['box'] :
                    fire_PO.loc[index, 'class0{}'.format(row['class'])] = 0
                    fire_PO.loc[index, 'PO'] = 0
                
                else :
                    fire_PO.loc[index, 'class0{}'.format(row['class'])] = row['polygon']
                    fire_PO.loc[index, 'PO'] = row['polygon']
                    
            else :
                 if row['class_count'] == row['box']:
                    fire_PO.loc[index, 'class{}'.format(row['class'])] = 0
                    fire_PO.loc[index, 'PO'] = 0

                 else :
                    fire_PO.loc[index, 'class{}'.format(row['class'])] = row['polygon']
                    fire_PO.loc[index, 'PO'] = row['polygon']

        # 빈 값은 0으로 대체
            fire_PO.fillna(0,inplace = True)
            
        fire_PO = fire_PO.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        fire_PO[fire_PO.columns.difference(['filename'])] = fire_PO[fire_PO.columns.difference(['filename'])].astype('int64')
        # csv 저장
        fire_PO.to_csv(main_path+ '/csv파일/PO/' +'{}PO.csv'.format(file_name, header = True, encoding = 'utf-8-sig', index = False))
    
        return fire_PO   

# 공사 DB
def insert_construct_po(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s2_po (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, class12, class13, class14, class15, class16, class17, class18, class19, class20, class21, class22, class23, class24, class25, class26, class27, class28, class29, class30, class31, class32, class33, class34, class35, class36, class37, class38, class39, class40, class41, class42, class43, class44, class45, PO) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

# 화재 DB
def insert_fire_po(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s3_po (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, PO) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) "
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

### 전체 DB

In [ ]:
# Making DB columns(공사)
def construct_DB(data):
        class_col = ['filename']

        for i in range(1, 46):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')
        class_col.append('PO')
        class_col.append('KP')

        construct_DB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

#             construct_DB.loc[index, '_id'] = index
            construct_DB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :

                construct_DB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    construct_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    construct_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    construct_DB.loc[index, 'KP'] = row['point']

            else :

                construct_DB.loc[index, 'class{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    construct_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    construct_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    construct_DB.loc[index, 'KP'] = row['point']

        # 빈 값은 0으로 대체
            construct_DB.fillna(0,inplace = True)
        
        construct_DB = construct_DB.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        construct_DB[construct_DB.columns.difference(['filename'])] = construct_DB[construct_DB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        construct_DB.to_csv(main_path+ '/csv파일/' +'{}.csv'.format(file_name, header = True, encoding = 'utf-8-sig', index = False))
        
        return construct_DB

# Making DB columns(화재)
def fire_DB(data):
        class_col = ['filename']

        for i in range(1, 11):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')
        class_col.append('PO')
        class_col.append('KP')

        fire_DB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

#             fire_DB.loc[index, '_id'] = index
            fire_DB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :

                fire_DB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    fire_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    fire_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    fire_DB.loc[index, 'KP'] = row['point']

            else :
                fire_DB.loc[index, 'class{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    fire_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    fire_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    fire_DB.loc[index, 'KP'] = row['point']

        # 빈 값은 0으로 대체
            fire_DB.fillna(0,inplace = True)
        
        fire_DB = fire_DB.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        fire_DB[fire_DB.columns.difference(['filename'])] = fire_DB[fire_DB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        fire_DB.to_csv(main_path+ '/csv파일/' +'{}.csv'.format(file_name, header = True, encoding = 'utf-8-sig', index = False))
    
        return fire_DB

# Making DB columns(항공)
def airport_DB(data):
    
        class_col = ['filename']
        for i in range(1, 23):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')
        class_col.append('PO')
        class_col.append('KP')

        airport_DB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

#             airport_DB.loc[index, '_id'] = index
            airport_DB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :

                airport_DB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    airport_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    airport_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    airport_DB.loc[index, 'KP'] = row['point']

            else :
                airport_DB.loc[index, 'class{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    airport_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    airport_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    airport_DB.loc[index, 'KP'] = row['point']

        # 빈 값은 0으로 대체
            airport_DB.fillna(0,inplace = True)
            
        airport_DB = airport_DB.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        airport_DB[airport_DB.columns.difference(['filename'])] = airport_DB[media_DB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        airport_DB.to_csv(main_path+ '/csv파일/' +'{}.csv'.format(file_name, header = True, encoding = 'utf-8-sig', index = False))
        
        return airport_DB
    
# 공사 DB
def insert_construct(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert ignore into s2 (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, class12, class13, class14, class15, class16, class17, class18, class19, class20, class21, class22, class23, class24, class25, class26, class27, class28, class29, class30, class31, class32, class33, class34, class35, class36, class37, class38, class39, class40, class41, class42, class43, class44, class45, BB, PO, KP) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

# 화재 DB
def insert_fire(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert ignore into s3 (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, BB, PO, KP) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()
            
# 항공 DB
def insert_airport(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert ignore into s1 (filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, class12, class13, class14, class15, class16, class17, class18, class19, class20, class21, class22, BB, PO, KP) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

### Set of Function

In [ ]:
# Making DB columns(공사)
def construct_DB(data):
        class_col = ['_id', 'filename']

        for i in range(1, 46):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')
        class_col.append('PO')
        class_col.append('KP')

        construct_DB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            construct_DB.loc[index, '_id'] = index
            construct_DB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :

                construct_DB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    construct_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    construct_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    construct_DB.loc[index, 'KP'] = row['point']

            else :

                construct_DB.loc[index, 'class{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    construct_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    construct_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    construct_DB.loc[index, 'KP'] = row['point']

        # 빈 값은 0으로 대체
            construct_DB.fillna(0,inplace = True)

        # 데이터 타입 변경
        construct_DB[media_DB.columns.difference(['filename'])] = construct_DB[construct_DB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        construct_DB.to_csv('{}.csv'.format(main_path.split('/')[-1]), header = True, encoding = 'utf-8-sig', index = False)
        
        return construct_DB

# Making DB columns(화재)
def fire_DB(data):
        class_col = ['_id', 'filename']

        for i in range(1, 11):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')
        class_col.append('PO')
        class_col.append('KP')

        fire_DB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            fire_DB.loc[index, '_id'] = index
            fire_DB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :

                fire_DB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    fire_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    fire_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    fire_DB.loc[index, 'KP'] = row['point']

            else :
                fire_DB.loc[index, 'class{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    fire_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    fire_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    fire_DB.loc[index, 'KP'] = row['point']

        # 빈 값은 0으로 대체
            fire_DB.fillna(0,inplace = True)

        fire_DB = fire_DB.groupby('filename',as_index=False).sum()
        # 데이터 타입 변경
        fire_DB[fire_DB.columns.difference(['filename'])] = fire_DB[fire_DB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        fire_DB.to_csv('{}.csv'.format(main_path.split('/')[-1]), header = True, encoding = 'utf-8-sig', index = False)
    
        return fire_DB

# Making DB columns(항공)
def airport_DB(data):
    
        class_col = ['_id', 'filename']
        for i in range(1, 23):
            if i < 10 :
                class_col.append('class0' + '{}'.format(i))
            else:
                class_col.append('class' + '{}'.format(i))
        class_col.append('BB')
        class_col.append('PO')
        class_col.append('KP')

        airport_DB = pd.DataFrame(columns=class_col)

        # DF에 저장
        for index, row in data.iterrows():

            airport_DB.loc[index, '_id'] = index
            airport_DB.loc[index, 'filename'] = row['filename']

            if row['class'] < 10 :

                airport_DB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    airport_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    airport_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    airport_DB.loc[index, 'KP'] = row['point']

            else :
                airport_DB.loc[index, 'class{}'.format(row['class'])] = row['class_count']

                if row['box'] != 0:
                    airport_DB.loc[index, 'BB'] = row['box']
                elif row['polygon'] != 0:
                    airport_DB.loc[index, 'PO'] = row['polygon']
                elif row['point'] != 0:
                    airport_DB.loc[index, 'KP'] = row['point']

        # 빈 값은 0으로 대체
            airport_DB.fillna(0,inplace = True)
        # 데이터 타입 변경
        airport_DB[airport_DB.columns.difference(['filename'])] = airport_DB[media_DB.columns.difference(['filename'])].astype('int64')
        # csv 저장
        airport_DB.to_csv('{}.csv'.format(main_path.split('/')[-1]), header = True, encoding = 'utf-8-sig', index = False)
        
        return airport_DB

In [ ]:
#전체 Select
def select_all():
    conn = pymysql.connect(host='localhost', user='root', password='1234', db='media', charset='utf8')
    try:
        with conn.cursor() as curs:
            sql = "select * from s2"
            curs.execute(sql)
            rs = curs.fetchall()
            for row in rs:
                print(row)
    finally:
        conn.close()

### DB구축

In [ ]:
# 공사 DB
def insert_construct(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s2 (_id, filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, class12, class13, class14, class15, class16, class17, class18, class19, class20, class21, class22, class23, class24, class25, class26, class27, class28, class29, class30, class31, class32, class33, class34, class35, class36, class37, class38, class39, class40, class41, class42, class43, class44, class45, BB, PO, KP) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

# 화재 DB
def insert_fire(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s3 (_id, filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, BB, PO, KP) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()
            
# 항공 DB
def insert_airport(test):
    conn = pymysql.connect(host='localhost', user='root', password='1234', database='media')

    try:
        with conn.cursor() as curs:
            sql = "insert into s1 (_id, filename, class01, class02, class03, class04, class05, class06, class07, class08, class09, class10, class11, class12, class13, class14, class15, class16, class17, class18, class19, class20, class21, class22, BB, PO, KP) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            data = [tuple(x) for x in test.values]
            curs.executemany(sql, data)
        conn.commit()
    finally:
            conn.close()

In [ ]:
insert_fire(fire_DB(sample))

In [ ]:
fire_DB(sample)

### Test

In [ ]:
# Making DB columns(공사)
class_col = ['_id', 'filename']

for i in range(1, 46):
    if i < 10 :
        class_col.append('class0' + '{}'.format(i))
    else:
        class_col.append('class' + '{}'.format(i))
class_col.append('BB')
class_col.append('PO')
class_col.append('KP')

media_DB = pd.DataFrame(columns=class_col)

# DF에 저장
for index, row in sample.iterrows():

    media_DB.loc[index, '_id'] = index
    media_DB.loc[index, 'filename'] = row['filename']

    if row['class'] < 10 :
        
        media_DB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']
        
        if row['box'] != 0:
            media_DB.loc[index, 'BB'] = row['box']
        elif row['polygon'] != 0:
            media_DB.loc[index, 'PO'] = row['polygon']
        elif row['point'] != 0:
            media_DB.loc[index, 'KP'] = row['point']

    else :
        
        media_DB.loc[index, 'class{}'.format(row['class'])] = row['class_count']
        
        if row['box'] != 0:
            media_DB.loc[index, 'BB'] = row['box']
        elif row['polygon'] != 0:
            media_DB.loc[index, 'PO'] = row['polygon']
        elif row['point'] != 0:
            media_DB.loc[index, 'KP'] = row['point']
            
# 빈 값은 0으로 대체
    media_DB.fillna(0,inplace = True)

# 데이터 타입 변경
media_DB[media_DB.columns.difference(['filename'])] = media_DB[media_DB.columns.difference(['filename'])].astype('int64')
media_DB.to_csv('{}.csv'.format(main_path.split('/')[-1]), header = True, encoding = 'utf-8-sig', index = False)

In [ ]:
# Making DB columns(화재)
class_col = ['_id', 'filename']

for i in range(1, 11):
    if i < 10 :
        class_col.append('class0' + '{}'.format(i))
    else:
        class_col.append('class' + '{}'.format(i))
class_col.append('BB')
class_col.append('PO')
class_col.append('KP')

media_DB = pd.DataFrame(columns=class_col)

# DF에 저장
for index, row in sample.iterrows():

    media_DB.loc[index, '_id'] = index
    media_DB.loc[index, 'filename'] = row['filename']

    if row['class'] < 10 :

        media_DB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']
        
        if row['box'] != 0:
            media_DB.loc[index, 'BB'] = row['box']
        elif row['polygon'] != 0:
            media_DB.loc[index, 'PO'] = row['polygon']
        elif row['point'] != 0:
            media_DB.loc[index, 'KP'] = row['point']

    else :
        media_DB.loc[index, 'class{}'.format(row['class'])] = row['class_count']
        
        if row['box'] != 0:
            media_DB.loc[index, 'BB'] = row['box']
        elif row['polygon'] != 0:
            media_DB.loc[index, 'PO'] = row['polygon']
        elif row['point'] != 0:
            media_DB.loc[index, 'KP'] = row['point']
            
# 빈 값은 0으로 대체
    media_DB.fillna(0,inplace = True)

# 데이터 타입 변경
media_DB[media_DB.columns.difference(['filename'])] = media_DB[media_DB.columns.difference(['filename'])].astype('int64')
media_DB.to_csv('{}.csv'.format(main_path.split('/')[-1]), header = True, encoding = 'utf-8-sig', index = False)

In [ ]:
# Making DB columns(항공)
class_col = ['_id', 'filename']

for i in range(1, 23):
    if i < 10 :
        class_col.append('class0' + '{}'.format(i))
    else:
        class_col.append('class' + '{}'.format(i))
class_col.append('BB')
class_col.append('PO')
class_col.append('KP')

media_DB = pd.DataFrame(columns=class_col)

# DF에 저장
for index, row in sample.iterrows():

    media_DB.loc[index, '_id'] = index
    media_DB.loc[index, 'filename'] = row['filename']

    if row['class'] < 10 :

        media_DB.loc[index, 'class0{}'.format(row['class'])] = row['class_count']
        
        if row['box'] != 0:
            media_DB.loc[index, 'BB'] = row['box']
        elif row['polygon'] != 0:
            media_DB.loc[index, 'PO'] = row['polygon']
        elif row['point'] != 0:
            media_DB.loc[index, 'KP'] = row['point']

    else :
        media_DB.loc[index, 'class{}'.format(row['class'])] = row['class_count']
        
        if row['box'] != 0:
            media_DB.loc[index, 'BB'] = row['box']
        elif row['polygon'] != 0:
            media_DB.loc[index, 'PO'] = row['polygon']
        elif row['point'] != 0:
            media_DB.loc[index, 'KP'] = row['point']
            
# 빈 값은 0으로 대체
    media_DB.fillna(0,inplace = True)

# 데이터 타입 변경
media_DB[media_DB.columns.difference(['filename'])] = media_DB[media_DB.columns.difference(['filename'])].astype('int64')
media_DB.to_csv('{}.csv'.format(main_path.split('/')[-1]), header = True, encoding = 'utf-8-sig', index = False)

In [ ]:
# 데이터 프레임 모든 행 삭제
media_DB.drop(media_DB.index, inplace=True)

### smaple test

In [ ]:
# #엑셀파일 DB Insert
# def insert_csv_to_db():
#     conn = pymysql.connect(host='localhost', user='root', password='1234', db='media', charset='utf8')
#     try:
#         with conn.cursor() as curs:
#             sql = 'insert into s1 values(%s, %s)'
 
#             wb = load_workbook('{0}.csv'.format(main_path.split('/')[-1], data_only=True))
#             ws = wb[main_path.split('/')[-1]]
 
#             iter_rows = iter(ws.rows)
#             next(iter_rows)
#             for row in iter_rows:
#                 curs.execute(sql, (row[0].value, row[1].value))
#             conn.commit()
#     finally:
#         conn.close()
#         wb.close()
 
 
# if __name__ == "__main__":
# #     delete_all()
#     insert_csv_to_db()
#     select_all()

In [ ]:
insert_construct(media_DB)

In [ ]:
insert_fire(media_DB)

In [ ]:
insert_airport(media_DB)